# Imports

In [2]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [3]:
np.random.seed(131)
initial_params = np.random.random([66])

INITIALIZATION_METHOD = 'Angle'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [4]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [5]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label == 2:
        x.append(example)
        y.append(-1)
    elif label == 7:
        x.append(example)
        y.append(1)

In [6]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [7]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [8]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [9]:
device = qml.device("default.qubit", wires=8)

In [10]:
def unitary(params, wire1, wire2):
    # qml.RZ(0, wires=wire1)
    qml.RY(params[0], wires=wire1)
    # qml.RZ(0, wires=wire1)
    # qml.RZ(0, wires=wire2)
    qml.RY(params[1], wires=wire2)
    # qml.RZ(0, wires=wire2)
    qml.CNOT(wires=[wire2, wire1])
    # qml.RZ(0, wires=wire1)
    qml.RY(params[2], wires=wire2)
    qml.CNOT(wires=[wire1, wire2])
    qml.RY(params[3], wires=wire2)
    qml.CNOT(wires=[wire2, wire1])
    # qml.RZ(0, wires=wire1)
    qml.RY(params[4], wires=wire1)
    # qml.RZ(0, wires=wire1)
    # qml.RZ(0, wires=wire2)
    qml.RY(params[5], wires=wire2)
    # qml.RZ(0, wires=wire2)

In [11]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(8), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(8), rotation='Y')

    # First layer
    unitary(params[0:6], 1, 2)
    unitary(params[6:12], 3, 4)
    unitary(params[12:18], 5, 6)

    # Second layer
    unitary(params[18:24], 0, 1)
    unitary(params[24:30], 2, 3)
    unitary(params[30:36], 4, 5)
    unitary(params[36:42], 6, 7)

    # Third layer
    unitary(params[42:48], 2, 5)

    # Fourth layer
    unitary(params[48:54], 1, 2)
    unitary(params[54:60], 5, 6)

    # Fifth layer
    unitary(params[60:66], 2, 5)

    # Measurement
    return qml.expval(qml.PauliZ(5))

## Circuit example

In [12]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.65015361 0.94810917 0.38802889 0.64129616 0.69051205 0.12660931
 0.23946678 0.25415707 0.42644165 0.83900255 0.74503365 0.38067928
 0.26169292 0.05333379 0.43689638 0.20897912 0.59441102 0.09890353
 0.22409353 0.5842624  0.95908107 0.20988382 0.66133746 0.50261295
 0.32029143 0.12506485 0.80688893 0.98696002 0.54304141 0.23132314
 0.60351254 0.17669598 0.88653747 0.58902228 0.72117264 0.27567029
 0.78811469 0.1326223  0.39971595 0.62982409 0.42404345 0.16187284
 0.52034418 0.6070413  0.5808057  0.82111597 0.98499188 0.93449492
 0.90305486 0.3380262  0.78324429 0.74373474 0.58058546 0.43266356
 0.66792795 0.23668741 0.45173663 0.91999741 0.96687301 0.76905057
 0.32671177 0.62283984 0.19160224 0.24832171 0.11683869 0.01032549]

Example features: [ 3.77071733  1.15194295  0.28018286  1.72589748 -0.92377076  0.44623263
 -2.19895432 -1.02122823]

Expectation value: 0.5810613569542123

 0: ──RY(3.77)────RY(0.224)──────────────────────────────────────────────────────────

# Accuracy test definition

In [13]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [14]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.5795053003533569
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.67962308598351
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.6972909305064783
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.7220259128386337
best accuracy so far!
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.7161366313309776
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.7302709069493523
best accuracy so far!
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.7809187279151943
best accuracy so far!
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.7573616018845701
epoch 71


epoch 344
epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.71849234393404
test_accuracies: [0.7809187279151943, 0.7573616018845701, 0.7561837455830389, 0.7608951707891637, 0.7809187279151943, 0.7667844522968198, 0.7302709069493523, 0.7220259128386337, 0.7573616018845701, 0.7762073027090695, 0.7585394581861014, 0.7832744405182568, 0.7585394581861014, 0.7726737338044758, 0.7773851590106007, 0.7797408716136631, 0.7773851590106007, 0.751472320376914, 0.7820965842167256, 0.7585394581861014, 0.7502944640753828, 0.7526501766784452, 0.7750294464075382, 0.7714958775029446, 0.7444051825677267, 0.7656065959952886, 0.7644287396937574, 0.7550058892815077, 0.7726737338044758, 0.71849234393404]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.7538280329799765
test_accuracies: [0.7573616018845701, 0.7561837455830389, 0.7608951707891637, 0.7809187279151943, 0.7667844522968198, 0.7302709069493523, 0.7220259128386337, 0

epoch 457
epoch 458
epoch 459
epoch 460
accuracy: 0.7691401648998822
test_accuracies: [0.7832744405182568, 0.7585394581861014, 0.7726737338044758, 0.7773851590106007, 0.7797408716136631, 0.7773851590106007, 0.751472320376914, 0.7820965842167256, 0.7585394581861014, 0.7502944640753828, 0.7526501766784452, 0.7750294464075382, 0.7714958775029446, 0.7444051825677267, 0.7656065959952886, 0.7644287396937574, 0.7550058892815077, 0.7726737338044758, 0.71849234393404, 0.7538280329799765, 0.7538280329799765, 0.740871613663133, 0.7396937573616018, 0.7255594817432274, 0.7479387514723204, 0.729093050647821, 0.7703180212014135, 0.7608951707891637, 0.7667844522968198, 0.7691401648998822]


In [15]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [ 1.76086958  1.42610744  0.35176263  1.59142967  0.79837387  0.23469539
  1.17745453  0.50719896  0.08364071 -0.31570917  0.70740526  0.4464531
  0.110462    0.68552761  0.88506912 -0.41812044  0.99718175  0.35007772
 -0.65727027  0.69212422  1.24360131  0.95157364  0.66133746  0.98909537
  0.42837752  0.08743646  0.09403169  0.22847851  0.82887836  0.23132314
  0.66928635  0.57946672  0.43146672  0.37098116  0.72117264 -0.03612114
  1.03928888  0.14283917  0.15527573  0.90815491  0.58747841  0.16187284
  0.80618112  0.29524987  1.27614533  2.06501807  1.54825253  0.47968239
  1.38953728  0.90128685  0.63479264  0.71803245  0.58058546  0.35738843
  0.21311542  0.40012237 -0.09019141 -0.05305491  1.21881179  0.76905057
  0.25143664  0.87477863 -0.04790005  0.50936946  0.11683869 -0.05987968]


# Testing

In [16]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.7638824078394774


In [17]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 2.8334217071533203
training time: 1741.7113196849823
test time: 54.76153087615967
total time: 1799.3062722682953
